In [36]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset
import os
from PIL import Image
import sklearn
import sklearn.metrics as sklm

In [52]:
!conda search torch

In [27]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
BATCH_SIZE = 32

data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because scale doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

In [11]:
class CustomDataset(Dataset):

    def __init__(self,path_to_images,labelcsv,transform=None):

        self.transform = transform
        self.path_to_images = path_to_images
        #self.df = pd.read_csv("nih_labels.csv")
        self.df = pd.read_csv(labelcsv)
        #self.df = self.df[self.df['fold'] == fold]

        self.df = self.df.set_index("Image Index")
        self.PRED_LABEL = [
            'Atelectasis',
            'Cardiomegaly',
            'Effusion',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pneumonia',
            'Pneumothorax',
            'Consolidation',
            'Edema',
            'Emphysema',
            'Fibrosis',
            'Pleural_Thickening',
            'Hernia']
        
        RESULT_PATH = "results/"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        image = Image.open(
            os.path.join(
                self.path_to_images,
                self.df.index[idx]))
        image = image.convert('RGB')

        label = np.zeros(len(self.PRED_LABEL), dtype=int)
        for i in range(0, len(self.PRED_LABEL)):
             # can leave zero if zero, else make one
            if(self.df[self.PRED_LABEL[i].strip()].iloc[idx].astype('int') > 0):
                label[i] = self.df[self.PRED_LABEL[i].strip()
                                   ].iloc[idx].astype('int')

        if self.transform:
            image = self.transform(image)

        return (image, label,self.df.index[idx])

In [32]:
def load_data(data_path=None):
    
    data_train = CustomDataset(
        path_to_images='/Users/gnsandeep/Documents/CS598/CS598Project/data/temp/train/train/',
        labelcsv = 'train_small_updated.csv',
        transform=data_transforms['train'])
    data_val = CustomDataset(
        path_to_images='/Users/gnsandeep/Documents/CS598/CS598Project/data/temp/val/val/',
        labelcsv = 'val_small_updated.csv', 
        transform=data_transforms['val'])
    
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=32, shuffle=False)
    
    return train_loader, val_loader , data_train , data_val

In [33]:
train_loader, val_loader , train_dataset , val_dataset = load_data()

assert type(train_loader) is torch.utils.data.dataloader.DataLoader
len(train_loader),len(val_loader)

(28, 8)

In [14]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3,20,5,1)
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.conv3 = nn.Conv2d(50,50,4,1)
        self.fc1 = nn.Linear(25*25*50,500)
        self.droupout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500,14)
        # your code here
        #raise NotImplementedError

    def forward(self, x):
        #input is of shape (batch_size=32, 3, 224, 224) if you did the dataloader right
        # your code here
        #raise NotImplementedError
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,25*25*50)
        x = F.relu(self.fc1(x))
        x = self.droupout1(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

    

In [15]:
def get_cnn_model():
    
    """
    TODO: Define the CNN model here. 
        We will use the pretrained ResNet18 model, which can be initialized with torchvision.models.resnet18
        Then, replace the last layer (model.fc) with a nn.Linear layer
            The new model.fc should have the same input size but a new output_size of 2
    """
    
    from torchvision import models
    
    num_classes = 14
    # your code here
    #raise NotImplementedError
    #old code model = torchvision.models.resnet18()
    model = models.densenet121(pretrained=True)
    #old code num_ftrs = model.fc.in_features
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, num_classes), nn.Sigmoid())
    # old code model.fc = nn.Linear(num_ftrs, num_classes)
    #For computation efficiency, we will freeze the weights in the bottom layers
    #If it's too slow, you can turn off layer4's weights update as well
    '''
    for param in model.named_parameters():
        if param[0].split(".")[0] in {'fc', 'layer4'}: continue
        param[1].requires_grad = False'''
    return model
    

In [16]:
#model = get_cnn_model()
model = SimpleCNN()
#model.load_state_dict(torch.load('resnet18_weights_9.pth', map_location='cpu'))

#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [23]:
n_epochs = 10

def train_model(model, train_dataloader, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim
    """
    :param model: A CNN model
    :param train_dataloader: the DataLoader of the training data
    :param n_epoch: number of epochs to train
    :return:
        model: trained model
    TODO:
        Within the loop, do the normal training procedures:
            pass the input through the model
            pass the output through loss_func to compute the loss (name the variable as *loss*)
            zero out currently accumulated gradient, use loss.basckward to backprop the gradients, then call optimizer.step
    """
    model.train() # prep model for training
    
    
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        #for data, target in train_dataloader:
        for data in train_dataloader:
            # your code here
            inputs, labels, _ = data
            optimizer.zero_grad()
            #y_hat = model(data)
            y_hat = model(inputs)
            labels = labels.type(torch.FloatTensor)
            #print(y_hat)
            #print(labels)
            #print(type(labels))
            


            #print((y_hat.shape))
            #print((target.shape))
            #loss = criterion(y_hat, target)
            loss = criterion(y_hat, labels)
            loss.backward()
            optimizer.step()
            #curr_epoch_loss.append(loss.item())
            #raise NotImplementedError
            curr_epoch_loss.append(loss.cpu().data.numpy())
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
    return model





In [24]:
train_loader, val_loader = load_data()
model = train_model(model, train_loader)

Epoch 0: curr_epoch_loss=0.6850053668022156
Epoch 1: curr_epoch_loss=0.6807374358177185
Epoch 2: curr_epoch_loss=0.676136314868927
Epoch 3: curr_epoch_loss=0.6704983115196228
Epoch 4: curr_epoch_loss=0.6638619303703308
Epoch 5: curr_epoch_loss=0.6556918025016785
Epoch 6: curr_epoch_loss=0.6447728872299194
Epoch 7: curr_epoch_loss=0.6303286552429199
Epoch 8: curr_epoch_loss=0.6097058057785034
Epoch 9: curr_epoch_loss=0.5802041888237


In [34]:
def eval_model(model, dataloader):
    """
    :return:
        Y_pred: prediction of model on the dataloder.
            Should be an 2D numpy float array where the second dimension has length 2.
        Y_test: truth labels. Should be an numpy array of ints
    TODO:
        evaluate the model using on the data in the dataloder.
        Add all the prediction and truth to the corresponding list
        Convert Y_pred and Y_test to numpy arrays (of shape (n_data_points, 2))
    """
    model.eval()
    #Y_pred = []
    #Y_test = []
    pred_df = pd.DataFrame(columns=["Image Index"])
    true_df = pd.DataFrame(columns=["Image Index"])
    #for data, target in dataloader:
    for i, data in enumerate(dataloader):    
        # your code here
        inputs, labels, _ = data
        true_labels = labels.detach().numpy()
        batch_size = true_labels.shape
        print("batch_size : " , batch_size)
        y_hat = model(inputs)
        probs = y_hat.detach().numpy()
        #y_hat = model(data)
        #y_hat_ = torch.max(y_hat,dim=1)
        #_, predicted = torch.max(y_hat, 1)
        #print(type(y_hat),y_hat)
        #print("predicted : " ,type(predicted),predicted.shape)
        #print("target : " ,target)
        #Y_pred.append(predicted.detach().numpy())
        #Y_test.append(target.detach().numpy())
        for j in range(0, batch_size[0]):
            thisrow = {}
            truerow = {}
            thisrow["Image Index"] = val_dataset.df.index[BATCH_SIZE * i + j]
            truerow["Image Index"] = val_dataset.df.index[BATCH_SIZE * i + j]

            # iterate over each entry in prediction vector; each corresponds to
            # individual label
            for k in range(len(val_dataset.PRED_LABEL)):
                thisrow["prob_" + val_dataset.PRED_LABEL[k]] = probs[j, k]
                truerow[val_dataset.PRED_LABEL[k]] = true_labels[j, k]

            pred_df = pred_df.append(thisrow, ignore_index=True)
            true_df = true_df.append(truerow, ignore_index=True)

        if(i % 10 == 0):
            print(str(i * BATCH_SIZE))
    
    #print()
        #raise NotImplementedError
    #Y_pred = np.concatenate(Y_pred, axis=0)
    #Y_test = np.concatenate(Y_test, axis=0)

    return pred_df, true_df
    

In [35]:
pred_df, true_df = eval_model(model, val_loader)


batch_size :  (32, 14)
0
batch_size :  (32, 14)
batch_size :  (32, 14)
batch_size :  (32, 14)
batch_size :  (32, 14)
batch_size :  (32, 14)
batch_size :  (32, 14)
batch_size :  (1, 14)


In [46]:
true_df.columns


Index(['Image Index', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
       'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'],
      dtype='object')

In [50]:
auc_df = pd.DataFrame(columns=["label", "auc"])

for column in true_df:
    #print('---------------------')
    #print("Column : " , column)
    if column not in [
            'Atelectasis',
            'Cardiomegaly',
            'Effusion',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pneumonia',
            'Pneumothorax',
            'Consolidation',
            'Edema',
            'Emphysema',
            'Fibrosis',
            'Pleural_Thickening',
                'Hernia']:
        continue
    actual = true_df[column]
    #print( 'Actual : ' , actual)
    pred = pred_df["prob_" + column]
    #print('Pred : ' , pred)
    thisrow = {}
    thisrow['label'] = column
    thisrow['auc'] = np.nan
    try:
        thisrow['auc'] = sklm.roc_auc_score(
        actual.values.astype(int), pred.values)
        #auc_df = auc_df.append(thisrow, ignore_index=True)
    #except Exception:
        
    except BaseException as e:
        print("can't calculate auc for " + str(column))
        print(e)
    auc_df = auc_df.append(thisrow, ignore_index=True)

pred_df.to_csv("preds.csv", index=False)
auc_df.to_csv("aucs.csv", index=False)


can't calculate auc for Hernia
Only one class present in y_true. ROC AUC score is not defined in that case.
